In [70]:
import pandas as pd
import numpy as np
from functools import reduce
from IPython.display import display
import re

In [71]:
vaccinationsDF = pd.read_csv('dataFiles/vaccinations.csv', sep=',', header=0, na_filter=False)
casesDF = pd.read_csv('dataFiles/covidCases.csv', sep=',', header=0, na_filter=False)
iso_codesDF = pd.read_csv('dataFiles/iso_codes.csv', sep=',', header=1, na_filter=False)

In [72]:
display(iso_codesDF)

,Country name,Official state name,Sovereignty,Alpha-2 code,Alpha-3 code,Numeric code,Subdivision code links,Internet ccTLD
0,Afghanistan,The Islamic Republic of Afghanistan,UN member state,AF,AFG,004,ISO 3166-2:AF,.af
1,"Akrotiri and Dhekelia – See United Kingdom, The.","Akrotiri and Dhekelia – See United Kingdom, The.","Akrotiri and Dhekelia – See United Kingdom, The.","Akrotiri and Dhekelia – See United Kingdom, The.","Akrotiri and Dhekelia – See United Kingdom, The.","Akrotiri and Dhekelia – See United Kingdom, The.","Akrotiri and Dhekelia – See United Kingdom, The.","Akrotiri and Dhekelia – See United Kingdom, The."
2,Åland Islands,Åland,Finland,AX,ALA,248,ISO 3166-2:AX,.ax
3,Albania,The Republic of Albania,UN member state,AL,ALB,008,ISO 3166-2:AL,.al
4,Algeria,The People's Democratic Republic of Algeria,UN member state,DZ,DZA,012,ISO 3166-2:DZ,.dz
...,...,...,...,...,...,...,...,...
275,Wallis and Futuna,The Territory of the Wallis and Futuna Islands,France,WF,WLF,876,ISO 3166-2:WF,.wf
276,Western Sahara,The Sahrawi Arab Democratic Republic,Disputed,EH,ESH,732,ISO 3166-2:EH,
277,Yemen,The Republic of Yemen,UN member state,YE,YEM,887,ISO 3166-2:YE,.ye
278,Zambia,The Republic of Zambia,UN member state,ZM,ZMB,894,ISO 3166-2:ZM,.zm


In [73]:
casesDF.rename(columns={'Date_reported':'date', 'Country':'location', 'Country_code': 'iso2Code'}, inplace=True)

vaccinationsDF.rename(columns={'iso_code': 'iso3Code'}, inplace=True)
vaccinationsDF.drop(vaccinationsDF[(vaccinationsDF['iso3Code'].apply(len) != 3)].index, inplace=True)
vaccinationsDF.reset_index(drop=True)

iso_codesDF.drop(iso_codesDF.columns[[1, 2, 5, 6, 7]], axis=1, inplace=True)
iso_codesDF.rename(columns={'Country name':'location', 'Alpha-2 code':'iso2Code', 'Alpha-3 code':'iso3Code'}, inplace=True)
iso_codesDF = iso_codesDF.astype(str)
iso_codesDF.drop(iso_codesDF[(iso_codesDF['iso2Code'].apply(len) != 2) | (iso_codesDF['iso3Code'].apply(len) != 3)].index, inplace=True)
iso_codesDF.reset_index(drop=True)

# Python czyta "NA" jako nan i sam to podmienia, wymagana ręczna naprawa.
# iso_codesDF.loc[iso_codesDF['location'].str.contains('Namibia'), 'iso2Code'] = 'NA'

,location,iso2Code,iso3Code
0,Afghanistan,AF,AFG
1,Åland Islands,AX,ALA
2,Albania,AL,ALB
3,Algeria,DZ,DZA
4,American Samoa,AS,ASM
...,...,...,...
244,Wallis and Futuna,WF,WLF
245,Western Sahara,EH,ESH
246,Yemen,YE,YEM
247,Zambia,ZM,ZMB


In [74]:
tmp_df = reduce(lambda left, right: pd.merge(left, right, on=['iso3Code'], how='inner'), [vaccinationsDF, iso_codesDF])
df = reduce(lambda left, right: pd.merge(left, right, on=['iso2Code', 'date'], how='outer'), [tmp_df, casesDF])

In [75]:
df = df.drop(['location_x', 'location_y', 'WHO_region', 'iso3Code'], axis=1)

In [ ]:
# display(casesDF['location'].where(casesDF['iso2Code'].isnull()).unique())
# display(casesDF['iso2Code'].where(casesDF['location'].isnull()).unique())
# display(df.loc[df['iso2Code'] == 'XA'])
# display(df.isnull().sum())
# display(df.count())
display(df[df['iso3Code'].isna()])

In [ ]:
"""
display(len(df.location_x.unique()))
display(len(df.location_y.unique()))
display(len(df.location.unique()))
# display(casesDF.tail(5))
# display(vaccinationsDF.tail(5))

# display(vaccinationsDF)
display(iso_codesDF.location.unique().size)
display(len(vaccinationsDF.iso3Code.unique()))
# display(df['location'].where(['iso2Code'].isnull()))
display(df.loc[df['location_x'].isnull()])"""
display(casesDF.count())


In [ ]:
def common_member(a, b):
    result = [i for i in a if i not in b]
    return len(result)

# display(len(common_member(casesDF.location.unique(), vaccinationsDF.location.unique())))

# display(common_member(casesDF.iso2Code.unique(), iso_codesDF.iso2Code.unique()))
# display(common_member(vaccinationsDF.iso3Code.unique(), iso_codesDF.iso3Code.unique()))
# display(casesDF.iso2Code.unique())
# display(iso_codesDF.loc[iso_codesDF['location'] == 'Namibia'])
display(common_member(df.location.unique(), df.location_x.unique()))

In [ ]:
df = reduce(lambda left, right: pd.merge(left, right, on=['date', 'iso2Code', 'iso3Code'], how='outer'), [casesDF, vaccinationsDF, iso_codesDF])
df = df.sort_values(by=['date', 'location'])
df = df.sort_values(by=['date', 'location'])
display(df.head(5))
df.to_csv('merged2.csv', header=True, sep=';')

In [ ]:
vaccinationsDF['date'] = pd.to_datetime(vaccinationsDF['date'], format='%Y-%m-%d').apply(lambda x: x.toordinal())
casesDF['Date_reported'] = pd.to_datetime(casesDF['Date_reported'], format='%Y-%m-%d').apply(lambda x: x.toordinal())